## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [3]:
!pip install --upgrade nltk gensim bokeh wget

  Obtaining dependency information for gensim from https://files.pythonhosted.org/packages/ad/97/b8253236dfedb9094f4273393a3fd03997da81f27f15822e56128da894ae/gensim-4.3.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for bokeh from https://files.pythonhosted.org/packages/84/60/33f23dbb15fcd785d6287bdd6fa8c9d836dacb40e21c14aad41a57a0ec0f/bokeh-3.2.2-py3-none-any.whl.metadata
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.2/24.0 MB 6.1 MB/s eta 0:00:04
   - -------------------------------------- 0.6/24.0 MB 7.6 MB/s eta 0:00:04
   --- ------------------------------------ 1.9/24.0 MB 15.4 MB/s eta 0:00:02
   ----- ---------------------------------- 3.5/24.0 MB 22.2 MB/s eta 0:00:01
   -------- ------------------------------- 5.4/24.0 MB 24.4 MB/s eta 0:00:01
   ----------- ------------------------

In [9]:
# download the data:
import wget
wget.download("https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1", r"quora.txt")
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

'quora.txt'

In [10]:
import numpy as np

data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [11]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [13]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(line.lower()) for line in data]

In [14]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [15]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [17]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [18]:
# now you can get word vectors !
model.get_vector('anything')

array([-2.8877223 , -0.70852774,  1.3364086 ,  2.5731606 ,  1.9409745 ,
        1.5494663 ,  2.3660984 , -4.7217016 ,  1.1422914 ,  2.8146043 ,
       -0.41273364,  1.3348513 ,  5.3914404 ,  1.2109859 ,  3.5717947 ,
       -0.59715337,  0.2020445 ,  0.21544571, -0.11877065, -3.0285988 ,
       -3.3012602 ,  0.20402744, -1.8317202 ,  0.681967  ,  0.92977154,
       -1.4032501 ,  0.06582113, -0.05086741, -0.3420523 ,  0.27460393,
       -0.3466856 ,  0.0449673 ], dtype=float32)

In [27]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

[('cheese', 0.8441256284713745),
 ('butter', 0.8039426803588867),
 ('toast', 0.7904372811317444),
 ('slice', 0.7709746956825256),
 ('chicken', 0.7550399899482727),
 ('salad', 0.7542903423309326),
 ('cake', 0.751861572265625),
 ('meat', 0.7513599395751953),
 ('soup', 0.747043251991272),
 ('dough', 0.7461358308792114)]

In [28]:
import numpy as np
np.dot(model.get_vector("bread"), model.get_vector("cheese")) / (np.linalg.norm(model.get_vector("cheese"), ord=2) * np.linalg.norm(model.get_vector("bread"), ord=2))

0.8441257

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [20]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [21]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155739784241),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.49220189452171326),
 ('shopper', 0.4920562207698822),
 ('part-time', 0.4912828207015991),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.4796452820301056)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [29]:
words = model.index_to_key[:1000] 

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [34]:
# for each word, compute it's vector with model
word_vectors = np.concatenate([model.get_vector(w).reshape(1, -1) for w in words], axis=0)
word_vectors.shape

(1000, 100)

In [35]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [46]:
from sklearn.decomposition import PCA

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
pca = PCA(n_components=2)
word_vectors_pca = pca.fit_transform(word_vectors)
word_vectors_pca = (word_vectors_pca - word_vectors_pca.mean(0)) / word_vectors_pca.std(0)

In [47]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [48]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [49]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

figure(id='p1094', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [50]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: don't panic it may take a minute or two to fit.
# normalize them as just lke with pca

tsne = TSNE()
word_tsne = tsne.fit_transform(word_vectors)
word_tsne = (word_tsne - word_tsne.mean(0)) / word_tsne.std(0)

In [51]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

figure(id='p1139', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [86]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    vector = np.zeros([model.vector_size], dtype='float32')
    
    tokens = tokenizer.tokenize(phrase.lower())
    vector = model.get_mean_vector(tokens, pre_normalize=False)
    
    return vector
        
    

In [87]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [88]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.concatenate([get_phrase_embedding(phrase).reshape(1, -1) for phrase in chosen_phrases], axis=0)

In [89]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [90]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE().fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

In [92]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

figure(id='p1319', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [93]:
# compute vector embedding for all lines in data
from tqdm.auto import tqdm
data_vectors = np.array([get_phrase_embedding(l) for l in tqdm(data)])
data_vectors.shape

  0%|          | 0/537272 [00:00<?, ?it/s]

(537272, 100)

In [143]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    vector = get_phrase_embedding(query)
    similiarity = (vector @ data_vectors.T) / (np.linalg.norm(vector) * np.linalg.norm(data_vectors, axis=1) + 1e-10)
    idxs = np.argsort(similiarity)
    return np.array(data)[list(idxs[-k:])]

In [144]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

What do I do to enter the line of event management?
How do I estimate the competition?
How do I use the pad in MMA?
How do I find the 5th dimension?
How do I think out of the box? How do I learn to think out of the box?
How do I win this?
What can I do to save the world?
How do I use the Greenify app?
What should I do to enter hollywood?
How do I get to the dark web?



AssertionError: 

In [137]:
find_nearest(query="How does Trump?", k=10)

array(['What does salesforce mean?\n',
       'How does it feel to date Ivanka Trump?\n',
       'What does Dushka Zapata think about Donald Trump?\n',
       'What companies does Donald Trump own?\n',
       'What does Donald Trump think about Pakistan?\n',
       'What does Donald Trump think of China?\n',
       'What does India think of Donald Trump?\n',
       'What does Donald Trump think of India?\n',
       'What books does Donald Trump like?\n',
       'What does Donald Trump think about Israel?\n'], dtype='<U1170')

In [138]:
find_nearest(query="Why don't i ask a question myself?", k=10)

array(["Why don't I have a girlfriend?\n",
       "Can I ask a girl out that I don't know?\n",
       "Why don't I ever get a girl?\n", "Why can't I take a joke?\n",
       "I don't have no question?\n", "Why don't I have a boyfriend?\n",
       "Why don't I get a girlfriend?\n",
       "Why can't I ask a question anonymously?\n",
       "Why do you always answer a question with a question? I don't, or do I?\n",
       "Why don't I get a date?\n"], dtype='<U1170')

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.